In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle
import random
import numpy as np
from asyncio import exceptions

In [3]:
def pickle_load(dir, filename):
  # open a file, where you stored the pickled data
  file = open(os.path.join(dir,filename), 'rb')

  # dump information to that file
  data = pickle.load(file)

  # close the file
  file.close()
  return data

In [4]:
dir = '/content/drive/MyDrive/CVPR/Project/Pickle 2'
training_data_x = pickle_load(dir, "training_data_x.pk")
training_data_y = pickle_load(dir, "training_data_y.pk")

testing_data_x = pickle_load(dir, "testing_data_x.pk")
testing_data_y = pickle_load(dir, "testing_data_y.pk")

In [5]:
from tensorflow.keras.utils import to_categorical
training_data_y = to_categorical(training_data_y, 4)
testing_data_y = to_categorical(testing_data_y, 4)

In [6]:
print(training_data_x.shape, training_data_y.shape)

print(testing_data_x.shape, testing_data_y.shape)


(500, 256, 256, 3) (500, 4)
(100, 256, 256, 3) (100, 4)


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
model = keras.Sequential([
    keras.Input(shape=(256, 256, 3)),
    layers.Conv2D(filters=112, kernel_size=(5,5),  activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Conv2D(filters=80, kernel_size=(3,3), strides=(1,1),  activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Conv2D(filters=128, kernel_size=(5,5), strides=(1,1),  activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Conv2D(filters=32, kernel_size=(5,5), strides=(1,1),  activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Flatten(),
    layers.Dense(units=768, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(units=4, activation='softmax')

])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 252, 252, 112)     8512      
                                                                 
 batch_normalization (BatchN  (None, 252, 252, 112)    448       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 112)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 80)      80720     
                                                                 
 batch_normalization_1 (Batc  (None, 124, 124, 80)     320       
 hNormalization)                                                 
                                                        

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [10]:
hist = model.fit(x=training_data_x, y = training_data_y, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
13/13 [==============================] - 17s 391ms/step - loss: 1.2235 - accuracy: 0.4600 - val_loss: 33.1267 - val_accuracy: 0.2800
Epoch 2/30
13/13 [==============================] - 3s 207ms/step - loss: 0.5037 - accuracy: 0.6575 - val_loss: 26.6272 - val_accuracy: 0.3200
Epoch 3/30
13/13 [==============================] - 3s 195ms/step - loss: 0.3501 - accuracy: 0.7100 - val_loss: 8.6383 - val_accuracy: 0.3500
Epoch 4/30
13/13 [==============================] - 3s 200ms/step - loss: 0.2960 - accuracy: 0.7700 - val_loss: 7.2719 - val_accuracy: 0.2700
Epoch 5/30
13/13 [==============================] - 3s 211ms/step - loss: 0.2516 - accuracy: 0.8050 - val_loss: 3.4869 - val_accuracy: 0.3800
Epoch 6/30
13/13 [==============================] - 3s 209ms/step - loss: 0.1639 - accuracy: 0.8950 - val_loss: 5.0879 - val_accuracy: 0.2000
Epoch 7/30
13/13 [==============================] - 3s 195ms/step - loss: 0.1523 - accuracy: 0.9150 - val_loss: 2.2030 - val_accuracy: 0.4100
Epo

In [11]:
model.evaluate(x=testing_data_x, y=testing_data_y, batch_size=32)

4/4 [==============================] - 0s 57ms/step - loss: 1.1291 - accuracy: 0.6300


[1.1290979385375977, 0.6299999952316284]